In [1]:
using DifferentialEquations
using Distributions
using LinearAlgebra
using Maybe
using Random
using Plots

# Soft Kilobot Model

In [2]:
VEL_X   = 0;
VEL_Y   = 1;
POS_X   = 2;
POS_Y   = 3;
FIELDS  = 4;

In [3]:
mutable struct SoftKilobot
    N_SIDE::Int               # Robots per side
    MASS::Float64             # Robot mass [kg]
    V_MAX::Float64            # Max speed
    V_DAMP::Float64           # Speed damping
    S_REST::Float64           # Spring rest length [m]
    S_STIFF::Float64          # Spring stiffness
    S_DAMP::Float64           # Spring damping
    N_FACTOR::Maybe.T{Normal} # Noise factor on velocity
    n::Int                    # Number of robots
    neighs::Array{Array{Int}} # Neighbors
    v_bias::Matrix{Float64}   # Speed bias
    v_old::Matrix{Float64}    # Old velocity
    SoftKilobot(n_side,mass,v_max,v_damp,s_rest,s_stiff,s_damp,n_factor) = (
        y = new(n_side,mass,v_max,v_damp,s_rest,s_stiff,s_damp,n_factor);
        y.n = n_side^2;
        y.neighs = [ Int[] for k in 1:y.n ];
        y.v_bias = zeros(2, y.n);
        y.v_old = zeros(2, y.n);
        return y)
end;

In [4]:
function SoftKilobot_Init(
        cntr::Vector{Float64},                   # Center of the robot distribution
        ornt::Float64,                           # Orientation of robot distribution (degrees)
        n_side::Int,                             # Robots per side
        mass::Float64,                           # Robot mass
        v_max::Float64,                          # Max speed
        v_damp::Float64,                         # Speed damping
        s_rest::Float64,                         # Spring rest length
        s_stiff::Float64,                        # Spring stiffness
        s_damp::Float64,                         # Spring damping
        n_factor::Maybe.T{Normal{Float64}},      # Noise factor on velocity
        n_bias_length::Maybe.T{Normal{Float64}}, # Noise on velocity bias vector length
        n_bias_angle::Maybe.T{Normal{Float64}})  # Noise on velocity bias vector angle
    # Create empty spring system
    ss = SoftKilobot(n_side,mass,v_max,v_damp,s_rest,s_stiff,s_damp,n_factor)
    # Add spring links
    for k in 1:ss.n
        # Horizontal
        kk = (k - 1) % n_side
        if(kk > 0)        push!(ss.neighs[k],k-1) end
        if(kk < n_side-1) push!(ss.neighs[k],k+1) end
        # Vertical
        kk = floor(Int, (k - 1) / n_side)
        if(kk > 0)        push!(ss.neighs[k],k-n_side) end
        if(kk < n_side-1) push!(ss.neighs[k],k+n_side) end
    end
    # Calculate rotation matrix
    ornt *= pi / 180.0
    rm = [cos(ornt) -sin(ornt); sin(ornt) cos(ornt)]
    # Add robots, the ids grow with positive x and negative y
    #    +x ->
    # -y 1 2 3
    #  | 4 5 6
    #  v 7 8 9
    x = zeros(FIELDS, ss.n)
    for i in 0:(n_side-1)
        for j in 0:(n_side-1)
            k = i + n_side * j + 1
            px = rm * (s_rest * [i, j]) + cntr
            x[:, k] = [0.0 0.0 px[1] px[2]]
            if n_factor != nothing
                @? v_bias = [ abs(rand(n_bias_length)); 0.0 ]
                @? v_bias_angle = rand(n_bias_angle)
                @? v_bias_rot = [ cos(v_bias_angle) -sin(v_bias_angle); sin(v_bias_angle) cos(v_bias_angle) ]
                @? ss.v_bias[:, k] = v_bias_rot * v_bias
            end
        end
    end
    return ss, x
end;

In [5]:
function SoftKilobot_CalcForces(u, ss)
    fs = zeros(2,ss.n)
    for k in 1:ss.n
        # Robot index in state vector to identify block
        idx1 = (k-1) * FIELDS + 1
        # Go through neighbors
        for n in ss.neighs[k]
            # Avoid useless recalculations
            if k < n
                # Robot index in state vector to identify block
                idx2 = (n-1) * FIELDS + 1
                # Get positions
                p1 = u[POS_X + idx1:POS_Y + idx1,:]
                p2 = u[POS_X + idx2:POS_Y + idx2,:]
                # Calculate relative vector
                p12 = p2 - p1
                # Calculate distance
                dist = norm(p12)
                # Calculate force contribution and add it
                f = ss.S_STIFF * (p12 / dist) * (dist - ss.S_REST)
                fs[:,k] += f
                fs[:,n] -= f
            end
        end
    end
    return fs
end;

In [6]:
function SoftKilobot_UpdateState(du, u, ss, v_tgt, fs)
    for k in 1:ss.n
        # Robot index in state vector to identify block
        idx = (k-1) * FIELDS + 1
        # State update
        du[VEL_X + idx] = 0.0
        if u[VEL_X + idx] > -ss.V_MAX && u[VEL_X + idx] < ss.V_MAX
            du[VEL_X + idx] = -ss.V_DAMP * u[VEL_X + idx] + (v_tgt[1] - u[VEL_X + idx]) / ss.MASS + fs[1,k] / ss.MASS
        end
        du[VEL_Y + idx] = 0.0
        if u[VEL_Y + idx] > -ss.V_MAX && u[VEL_Y + idx] < ss.V_MAX
            du[VEL_Y + idx] = -ss.V_DAMP * u[VEL_Y + idx] + (v_tgt[2] - u[VEL_Y + idx]) / ss.MASS + fs[2,k] / ss.MASS
        end
        ss.v_old[1,k] = clamp(u[VEL_X + idx] + ss.v_bias[1,k], -ss.V_MAX, ss.V_MAX)
        ss.v_old[2,k] = clamp(u[VEL_Y + idx] + ss.v_bias[2,k], -ss.V_MAX, ss.V_MAX)
        du[POS_X + idx] = ss.v_old[1,k]
        du[POS_Y + idx] = ss.v_old[2,k]
    end
    return du
end;

In [7]:
function SoftKilobot_Step!(du, u, ss, t)
    # Calculate forces
    fs = SoftKilobot_CalcForces(u, ss)
    # Calculate target speeds using algorithm
    # TODO
    v_tgt = [0.1; 0.0]
    # Update states
    du = SoftKilobot_UpdateState(du, u, ss, v_tgt, fs)
    # All done
    return du
end;

# Plotting

In [8]:
PLOT_SIDE  = 2.0;
PLOT_START = 0.4;
PLOT_END   = 1.6;

In [9]:
function SoftKilobot_Plot(x::Matrix{Float64}, t::Float64; vel=false, springs=false)
    # General settings
    p = plot(xlims=(0,2), ylims=(0,2), title=string("t = ", t), legend=:none)
    # Start and end lines
    plot!(p, [PLOT_START PLOT_END; PLOT_START PLOT_END], [0 0; PLOT_SIDE PLOT_SIDE])
    # Robots as circles
    scatter!(p, x[POS_X+1,:], x[POS_Y+1,:])
    # Plot velocities
    if(vel)
        for i in 1:ss.n
            plot!(p, [x[POS_X+1,i]; (x[POS_X+1,i]+x[VEL_X+1,i])], [x[POS_Y+1,i]; (x[POS_Y+1,i]+x[VEL_Y+1,i])])
            plot!(p, [x[POS_X+1,i]; (x[POS_X+1,i]+ss.v_bias[1,i])], [x[POS_Y+1,i]; (x[POS_Y+1,i]+ss.v_bias[2,i])])
            plot!(p, [x[POS_X+1,i]; (x[POS_X+1,i]+ss.v_old[1,i])], [x[POS_Y+1,i]; (x[POS_Y+1,i]+ss.v_old[2,i])])
        end
    end
    # Plot springs as segments
    if(springs)
        for i in 1:ss.n
            for j in ss.neighs[i]
                if(i < j)
                    plot!(p, [x[POS_X+1,i]; x[POS_X+1,j]], [x[POS_Y+1,i]; x[POS_Y+1,j]])
                end
            end
        end
    end
    return p
end;

# Demo Without Noise

In [10]:
# Control frequency
DELTA_T = 1.0 / 31.0;

In [ ]:
# Duration in seconds
DURATION = 120;

# Initialization
ss, u0 = SoftKilobot_Init(
    [0.4,1], # Center of the robot distribution
    135.0,   # Orientation of robot distribution (degrees)
    7,       # Robots per side
    0.024,   # Robot mass
    0.015,   # Max speed
    0.01,    # Speed damping
    0.031,   # Spring rest length
    0.6,     # Spring stiffness
    0.1,     # Spring damping
    nothing, # Noise factor on velocity
    nothing, # Noise on velocity bias vector length
    nothing) # Noise on velocity bias vector angle

# Execution
u = vec(u0)
@gif for i in 0:(DURATION/DELTA_T)
    tspan = (i * DELTA_T, (i+1) * DELTA_T);
    prob = ODEProblem(SoftKilobot_Step!, u, tspan, ss);
    sol = solve(prob)
    for j in 1:length(sol.t)
        SoftKilobot_Plot(reshape(sol.u[j], FIELDS, ss.n), sol.t[j])
    end
    u = sol.u[end]
end

# Demo With Noise

In [ ]:
# Duration in seconds
DURATION = 120;

Random.seed!(123)

# Initialization
ss, u0 = SoftKilobot_Init(
    [0.4,1],                          # Center of the robot distribution
    135.0,                            # Orientation of robot distribution (degrees)
    7,                                # Robots per side
    0.024,                            # Robot mass
    0.015,                            # Max speed
    0.01,                             # Speed damping
    0.031,                            # Spring rest length
    0.6,                              # Spring stiffness
    0.1,                              # Spring damping
    Some(Normal(0.9,0.1)),            # Noise factor on velocity
    Some(Normal(0.005,0.005)),        # Noise on velocity bias vector length
    Some(Normal(0.0, 45.0*pi/180.0))) # Noise on velocity bias vector angle

# Execution
u = vec(u0)
@gif for i in 0:(DURATION/DELTA_T)
    tspan = (i * DELTA_T, (i+1) * DELTA_T);
    prob = ODEProblem(SoftKilobot_Step!, u, tspan, ss);
    sol = solve(prob)
    for j in 1:length(sol.t)
        SoftKilobot_Plot(reshape(sol.u[j], FIELDS, ss.n), sol.t[j], vel=false, springs=false)
    end
    u = sol.u[end]
end